In [2]:
import cdt
import numpy as np
import pandas as pd

from causalgraphicalmodels import StructuralCausalModel
print(cdt.SETTINGS.GPU)

1


# Causal Models

In [3]:
def mu():
    return np.random.uniform() * 10

def sd():
    return np.random.uniform() * 2

In [4]:
# Linear model with uniform noise
linear_uniform = StructuralCausalModel({
    "X": lambda n_samples: np.random.normal(mu(), sd(), size=(n_samples, )),
    "Y": lambda X, n_samples: 3 * X + np.random.uniform(-1, 1, size=(n_samples, ))
})

# Linear model with Gaussian noise
linear_gaussian = StructuralCausalModel({
    "X": lambda n_samples: np.random.normal(mu(), sd(), size=(n_samples, )),
    "Y": lambda X, n_samples: 3 * X + np.random.normal(0, 1, size=(n_samples, ))
})

# Exponential model with uniform noise
exp_uniform = StructuralCausalModel({
    "X": lambda n_samples: np.random.normal(mu(), sd(), size=(n_samples, )),
    "Y": lambda X, n_samples: np.exp(X) + np.random.uniform(-1, 1, size=(n_samples, ))
})

# Exponential model with Gaussian noise
exp_gaussian = StructuralCausalModel({
    "X": lambda n_samples: np.random.normal(mu(), sd(), size=(n_samples, )),
    "Y": lambda X, n_samples: np.exp(X) + np.random.normal(0, 1, size=(n_samples, ))
})

# Logarithmic model with uniform noise
log_uniform = StructuralCausalModel({
    "X": lambda n_samples: np.random.normal(mu(), sd(), size=(n_samples, )),
    "Y": lambda X, n_samples: np.log(X) + np.random.uniform(-1, 1, size=(n_samples, ))
})

# Logarithmic model with Gaussian noise
log_gaussian = StructuralCausalModel({
    "X": lambda n_samples: np.random.normal(mu(), sd(), size=(n_samples, )),
    "Y": lambda X, n_samples: np.log(X) + np.random.normal(0, 1, size=(n_samples, ))
})

In [5]:
def get_data_samples(causal_model, n_samples):
    df = causal_model.sample(n_samples).dropna()
    X = df['X'].values
    Y = df['Y'].values
    return dict(X=X, Y=Y)

In [6]:
allowed_models = ['linear_uniform', 'linear_gaussian', 'exp_uniform', 'exp_gaussian']

models_dict = {k: v for k, v in locals().items() if isinstance(v, StructuralCausalModel) and k in allowed_models}

In [7]:
data = []

for i in range(10):
    for name, model in models_dict.items():
        model_sample = get_data_samples(model, n_samples=1000)
        model_sample['name'] = name
        data.append(model_sample)
        
        
df = pd.DataFrame(data)
data = df[['X', 'Y']].copy()

# Causal Discovery Toolbox

## ANM

ANM algorithm.

**Description:** The Additive noise model is one of the most popular approaches for pairwise causality. It bases on the fitness of the data to the additive noise model on one direction and the rejection of the model on the other direction.

**Data Type:** Continuous

**Assumptions:** Assuming that x→y
then we suppose that the data follows an additive noise model, i.e. y=f(x)+E. E being a noise variable and f a deterministic function. The causal inference bases itself on the independence between x and e. It is proven that in such case if the data is generated using an additive noise model, the model would only be able to fit in the true causal direction.

https://papers.nips.cc/paper/3548-nonlinear-causal-discovery-with-additive-noise-models.pdf

In [8]:
from cdt.causality.pairwise import ANM

In [9]:
anm = ANM()

In [10]:
y_pred = anm.predict(data)

In [11]:
df['y_pred_anm'] = np.array(y_pred) > 0
anm_result = df.groupby(['name', 'y_pred_anm']).size().unstack().fillna(0)
display(anm_result)

y_pred_anm,False,True
name,,
exp_gaussian,0.0,10.0
exp_uniform,0.0,10.0
linear_gaussian,5.0,5.0
linear_uniform,3.0,7.0


## Bivariate Fit

Bivariate Fit model.

**Description:** The bivariate fit model is based onon a best-fit criterion relying on a Gaussian Process regressor. Used as weak baseline.

**Data Type:** Continuous

**Assumptions:** This is often a model used to show that correlation ≠ causation. It holds very weak performance, as it states that the best predictive model is the causal model.

In [12]:
from cdt.causality.pairwise import BivariateFit

In [13]:
bvf = BivariateFit()

In [14]:
y_pred = bvf.predict(data)

In [15]:
df['y_pred_bvf'] = np.array(y_pred) > 0
bvf_result = df.groupby(['name', 'y_pred_bvf']).size().unstack().fillna(0)
display(bvf_result)

y_pred_bvf,False,True
name,,
exp_gaussian,0.0,10.0
exp_uniform,0.0,10.0
linear_gaussian,5.0,5.0
linear_uniform,7.0,3.0


## CDS

Conditional Distribution Similarity Statistic

**Description:** The Conditional Distribution Similarity Statistic measures the std. of the rescaled values of y (resp. x) after binning in the x (resp. y) direction. The lower the std. the more likely the pair to be x->y (resp. y->x). It is a single feature of the Jarfo model.

**Data Type:** Continuous and Discrete

**Assumptions:** This approach is a statistical feature of the joint distribution of the data mesuring the variance of the marginals, after conditioning on bins.

Fonollosa, José AR, “Conditional distribution variability measures for causality detection”, 2016.

In [17]:
from cdt.causality.pairwise import CDS

In [18]:
cds = CDS()

In [19]:
y_pred = cds.predict(data)

In [20]:
df['y_pred_cds'] = np.array(y_pred) > 0
cds_result = df.groupby(['name', 'y_pred_cds']).size().unstack().fillna(0)
display(cds_result)

y_pred_cds,False,True
name,,
exp_gaussian,4,6
exp_uniform,4,6
linear_gaussian,7,3
linear_uniform,4,6


## GNN

Shallow Generative Neural networks.

**Description:** Pairwise variant of the CGNN approach, Models the causal directions x->y and y->x with a 1-hidden layer neural network and a MMD loss. The causal direction is considered as the best-fit between the two causal directions.

**Data Type:** Continuous

**Assumptions:** The class of generative models is not restricted with a hard contraint, but with the hyperparameter nh. This algorithm greatly benefits from bootstrapped runs (nruns >=12 recommended), and is very computationnally heavy. GPUs are recommended.

Learning Functional Causal Models with Generative Neural Networks Olivier Goudet & Diviyan Kalainathan & Al. (https://arxiv.org/abs/1709.05321)

In [21]:
from cdt.causality.pairwise import GNN

In [22]:
gnn = GNN()

In [ ]:
## Takes a while to run (even on a GPU)
y_pred = gnn.predict(data)

In [ ]:
df['y_pred_gnn'] = np.array(y_pred) > 0
gnn_result = df.groupby(['name', 'y_pred_gnn']).size().unstack().fillna(0)
display(gnn_result)

## IGCI

IGCI model.

**Description:** Information Geometric Causal Inference is a pairwise causal discovery model model considering the case of minimal noise Y=f(X)
, with f invertible and leverages assymetries to predict causal directions.

**Data Type:** Continuous

**Assumptions:** Only the case of invertible functions only is considered, as the prediction would be trivial otherwise if the noise is minimal.

In [24]:
from cdt.causality.pairwise import IGCI

In [25]:
igci = IGCI()

In [26]:
y_pred = igci.predict(data)

In [27]:
df['y_pred_igci'] = np.array(y_pred) > 0
igci_result = df.groupby(['name', 'y_pred_igci']).size().unstack().fillna(0)
display(igci_result)

y_pred_igci,False,True
name,,
exp_gaussian,0.0,10.0
exp_uniform,0.0,10.0
linear_gaussian,5.0,5.0
linear_uniform,5.0,5.0


## Jarfo

Jarfo model, 2nd of the Cause Effect Pairs challenge, 1st of the Fast Causation Challenge.

**Description:** The Jarfo model is an ensemble method for causal discovery: it builds lots of causally relevant features (such as ANM) with a gradient boosting classifier on top.

**Data Type:** Continuous, Categorical, Mixed

**Assumptions:** _This method needs a substantial amount of labelled causal pairs to train itself_. Its final performance depends on the training set used.